# Lab 3 – Error‑Spotting & Self‑Feedback

### Learning Objectives
* Prompt the LLM to critique **its own** answer.
* Chain a self‑correction loop until quality thresholds pass.

# Lab 3 – Building Self‑Correcting Loops

In [ ]:
!pip -q install openai --progress-bar off
import openai, getpass, json, time, pprint, os

openai.api_key = os.getenv("OPENAI_API_KEY") or getpass.getpass()

QUESTION = "List three capital cities in South America."
EVAL_TEMPLATE = """Evaluate the answer strictly for factual accuracy. 
Respond with JSON: {{"valid": true/false, "issues": [], "fixed_answer": ""}}."""

def iterate(question, max_loops=3):
    answer = None
    for i in range(max_loops):
        answer = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":question}]).choices[0].message.content

        review = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[{"role":"user","content":EVAL_TEMPLATE+"\n\nAnswer:\n"+answer}]).choices[0].message.content
        data = json.loads(review)
        if data["valid"]:
            return data["fixed_answer"] or answer
        else:
            question = data["issues"][0] + " Please try again."
    return answer

print(iterate(QUESTION))

### Discussion

* How many loops before diminishing returns?  
* Where would you insert a **human reviewer**?